In [1]:
import os
os.chdir("/Users/Kuba/Library/CloudStorage/OneDrive-SGH/SGH/Rok IV/Analiza danych w czasie rzeczywistym/internet-of-trams") ### This needs to be replaced.

from internet_of_trams.api.database_connector import DatabaseConnector

In [64]:
lines = ["17", "18", "19"]

In [67]:
lines_str = ", ".join(lines)
lines_str = f"({lines_str})"
lines_str

'(17, 18, 19)'

In [21]:
connector = DatabaseConnector(username = 'root', password = 'my-secret-pw', host = '127.0.0.1')

query = "DROP SCHEMA internet_of_trams"

connector.execute(query)

NameError: name 'DatabaseConnector' is not defined

In [73]:
from datetime import datetime
import pytz
from dateutil.relativedelta import relativedelta


observation_cutoff = datetime.now() - relativedelta(minutes=5)
observation_cutoff_str = observation_cutoff.strftime("%Y%m%d%H%m%S")
extraction_cutoff = datetime.now() - relativedelta(minutes=2)
extraction_cutoff_str = extraction_cutoff.strftime("%Y%m%d%H%m%S")

connector = DatabaseConnector(username = 'root', password = 'my-secret-pw', host = '127.0.0.1', database="internet_of_trams")

query = f"""
    SELECT 
        v.id
        ,v.line_id
        ,v.type
        ,v.brigade
        ,a.longitude
        ,a.latitude
        ,a.timestamp
        ,a._extraction_timestamp
    FROM 
        appearance a 
    JOIN 
        vehicle v 
    ON 
        a.vehicle_id=v.id 
    WHERE 
        a.timestamp > STR_TO_DATE({observation_cutoff_str}, '%Y%m%d%H%i%s')
        AND a._extraction_timestamp > STR_TO_DATE({extraction_cutoff_str}, '%Y%m%d%H%i%s')
        AND v._extraction_timestamp > STR_TO_DATE({extraction_cutoff_str}, '%Y%m%d%H%i%s')
        AND a._extraction_timestamp = (SELECT MAX(s._extraction_timestamp) FROM appearance s WHERE a.vehicle_id = s.vehicle_id)
        AND v.line_id IN (17)
    """

connector.execute(query)

,id,line_id,type,brigade,longitude,latitude,timestamp,_extraction_timestamp


In [54]:
x

[{'id': 3812,
  'line_id': '20',
  'type': 'tram',
  'brigade': 20,
  'longitude': 20.938416,
  'latitude': 52.2467,
  'timestamp': Timestamp('2024-05-19 21:06:54'),
  '_extraction_timestamp': Timestamp('2024-05-19 21:07:14')},
 {'id': 3117,
  'line_id': '9',
  'type': 'tram',
  'brigade': 9,
  'longitude': 21.000511,
  'latitude': 52.18628,
  'timestamp': Timestamp('2024-05-19 21:07:30'),
  '_extraction_timestamp': Timestamp('2024-05-19 21:07:48')},
 {'id': 3126,
  'line_id': '9',
  'type': 'tram',
  'brigade': 9,
  'longitude': 21.01952,
  'latitude': 52.263386,
  'timestamp': Timestamp('2024-05-19 21:07:28'),
  '_extraction_timestamp': Timestamp('2024-05-19 21:07:48')},
 {'id': 3130,
  'line_id': '3',
  'type': 'tram',
  'brigade': 3,
  'longitude': 20.984417,
  'latitude': 52.218838,
  'timestamp': Timestamp('2024-05-19 21:07:28'),
  '_extraction_timestamp': Timestamp('2024-05-19 21:07:48')},
 {'id': 3133,
  'line_id': '9',
  'type': 'tram',
  'brigade': 9,
  'longitude': 20.980097

In [ ]:
WHERE a.timestamp > STR_TO_DATE({observation_cutoff_str}, '%Y%m%d%h%i%s')

In [28]:
datetime.now() - relativedelta(minutes=5)

datetime.datetime(2024, 5, 19, 20, 49, 10, 925155)

In [ ]:
 WHERE timestamp > STR_TO_DATE({observation_cutoff_str}, '%Y%m%d%h%i%s')

In [16]:
observation_cutoff.strftime("%Y-%m-%d %H-%m-%S")

'2024-05-19 18-05-18'

In [ ]:
def extract_appearances_of_vehicles_for_lines(self, lines):
    def extract_appearances_of_vehicles_for_line(vehicle_type, line, connector):
        URL = "https://api.um.warszawa.pl/api/action/busestrams_get/"

        type_mapping = {"bus": 1, "tram": 2}
        if vehicle_type not in type_mapping:
            raise ValueError("Vehicle type must be either tram or bus.")

        params = {
            "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59"
            ,"type": type_mapping[vehicle_type]
            ,"line": line
        }
        
        try:
            appearances_of_vehicles_for_line = connector.get(URL, params)
        except Exception as e:
            logging.warning(e)
            return []
        
        for aov in appearances_of_vehicles_for_line:
            aov["_extraction_timestamp"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return appearances_of_vehicles_for_line
    
    VEHICLE_TYPE = "tram"
    
    appearances_of_vehicles = []
    
    for line in lines:
        appearances_of_vehicles += extract_appearances_of_vehicles_for_line(VEHICLE_TYPE, line, self.__connector)
        
    return appearances_of_vehicles

In [ ]:
def get_vehicles_and_appearances(self, lines):
    def parse_appearances(appearances_of_vehicles):
        appearances = []
            
        for aov in appearances_of_vehicles:
            appearances.append(parse_appearance(aov))
            
        return appearances
    
    def parse_vehicles(appearances_of_vehicles):
        vehicles = []
            
        for aov in appearances_of_vehicles:
            vehicles.append(parse_vehicle(aov))
            
        return vehicles
    
    appearances_of_vehicles = self.extract_appearances_of_vehicles_for_lines(lines)
        
    self.vehicles = parse_vehicles(appearances_of_vehicles)
    self.appearances = parse_appearances(appearances_of_vehicles)

In [8]:
connector = DatabaseConnector(username = 'root', password = 'my-secret-pw', host = '127.0.0.1')

query = "SHOW SCHEMAS"

print(connector.execute(query))

             Database
0  information_schema
1   internet_of_trams
2               mysql
3  performance_schema
4                 sys


In [12]:
import os
os.environ["DATABASE_HOST"] = "1"

In [19]:
import os
import yaml

def get_config():
    config_path = "/Users/Kuba/Library/CloudStorage/OneDrive-SGH/SGH/Rok IV/Analiza danych w czasie rzeczywistym/internet-of-trams/configuration.yaml"
    
    with open(config_path) as config_file:
        config = yaml.safe_load(config_file)
        
    for key in config.keys():
        if os.getenv(key):
            config[key] = os.getenv(key)
        
    return config

In [15]:
for key in config.keys():
    if os.getenv(key):
        config[key] = os.getenv(key)

In [18]:
config

{'DATABASE_USERNAME': 'root',
 'DATABASE_HOST': '1',
 'DATABASE_PASSWORD': 'my-secret-pw',
 'API_KEY': '12b8f222-5689-4177-9ac2-01ff1229c098',
 'LINES': [1,
  2,
  3,
  4,
  6,
  7,
  9,
  10,
  11,
  13,
  15,
  17,
  18,
  20,
  22,
  23,
  24,
  26,
  27,
  28,
  31,
  33,
  35],
 'KAFKA_TOPIC': 'abc'}

In [20]:
config = get_config()
config

{'DATABASE_USERNAME': 'root',
 'DATABASE_HOST': '1',
 'DATABASE_PORT': 3306,
 'DATABASE_PASSWORD': 'my-secret-pw',
 'API_KEY': None,
 'LINES': [1,
  2,
  3,
  4,
  6,
  7,
  9,
  10,
  11,
  13,
  15,
  17,
  18,
  20,
  22,
  23,
  24,
  26,
  27,
  28,
  31,
  33,
  35],
 'KAFKA_TOPIC': 'abc'}

In [3]:
import os
x = os.getenv("DEBUSSY")
x

'1'

In [5]:
if os.getenv("DB"):
    print("a")
else:
    print("b")

b


In [ ]:
import subprocess

def run_backend():
    
    MODEL_REGISTRATOR_FILE_NAME = "register_models.py"
    LINES_AND_DESTINATIONS_UPLOADER_FILE_NAME = "upload_lines_and_destinations.py"
    STOPS_AND_POLES_UPLOADER_FILE_NAME = "upload_stops_and_poles.py"
    APPEARANCES_OF_VEHICLES_UPLOADER_FILE_NAME = "produce_appearances_of_vehicles.py"

    try:
        subprocess.run(["python", MODEL_REGISTRATOR_FILE_NAME])
        model_registrator = subprocess.run(["python", MODEL_REGISTRATOR_FILE_NAME])
        producer = subprocess.Popen(["python", PRODUCER_FILE_NAME])
        
        registrator.wait()
        producer.wait()
        
    except KeyboardInterrupt:
        registrator.kill()
        producer.kill()
        
def main():
    upload_appearances_of_vehicles()


if __name__ == "__main__":
    main()
